In [1]:
from time import sleep, time
from multiprocessing import Pool, cpu_count
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import datetime
import json
import re

In [2]:
namepackages = []

def get_url_packages(url):
    print("Scraping: ", url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find( "table" )

    for row in table.findAll("a"):

        name = row.text
        url = row.get('href').replace("../../", "https://cran.r-project.org/")

        namepackages.append({"name": name, "url": url})
    return namepackages

In [3]:
start_time = time()

url = ['https://cran.r-project.org/web/packages/available_packages_by_name.html']

with Pool(cpu_count()-1) as p:
    urls_packages = p.map(get_url_packages, url)
p.close()
p.join()
end_time = time()
elapsed_time = end_time - start_time

print(f'Elapsed run time: {elapsed_time} seconds')


Scraping:  https://cran.r-project.org/web/packages/available_packages_by_name.html
Elapsed run time: 2.045599937438965 seconds


In [4]:
for pkg in urls_packages:
    df = pd.DataFrame.from_dict(pkg)

In [5]:
df.head()

,name,url
0,A3,https://cran.r-project.org/web/packages/A3/ind...
1,aaSEA,https://cran.r-project.org/web/packages/aaSEA/...
2,ABACUS,https://cran.r-project.org/web/packages/ABACUS...
3,abbyyR,https://cran.r-project.org/web/packages/abbyyR...
4,abc,https://cran.r-project.org/web/packages/abc/in...


In [6]:
listPackages = df["url"].tolist()
len(listPackages)

15368

In [7]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [8]:
import pymongo

mongo = pymongo.MongoClient("mongodb://localhost:27017/")
database = mongo["PY_CRAN"]
collection = database["R_packages"]

In [9]:
import asyncio, re
from aiohttp import ClientSession
loop = asyncio.get_event_loop()

chr_replace = { ":": "", "\n": "", " ": "", "\t": ""}
MAX_SIM_CONNS = 50

async def fetch(url, session):
    async with session.get(url) as response:
         text = await response.read()
            
    data = {}
    soup = BeautifulSoup(text, "lxml")
        
    title = soup.find("h2").text
    description = soup.select("body > p")[0].text
    title_s = title.split(":")
    
    data['Package'] = replace_all(title_s[0].strip(), chr_replace)
    data['Title'] = replace_all(title_s[1].strip(), chr_replace)
    data['Description'] = replace_all(description.strip(), chr_replace)
    
    for table in soup.findAll('table'):
        rows = table.findAll("tr")
        for i in range(len(rows)):
            td_list = rows[i].findAll('td')

            name = "_".join(replace_all(td_list[0].text.strip(), chr_replace).split())
            value = replace_all(td_list[1].text.strip(), chr_replace).strip()
            
            #(?!<(?:\<|\[|\(|\")[^"\)\]\>]+),(?![^.\"\(\[\<]+(?:\.|\"|\>|\)|\]|\>)) #Primera Prueba
            #(?!<(?:\<|\"|\[|\()[^)\]\"\>]+),(?![^(\[\"\<]+(?:\>|\"|\)|\]|\>)) # Segunda Prueba
            #(?!<(?:\<|\"|\[|\()[^)\]\"\>]+)[,|;](?![^(\[\"\<]+(?:\>|\"|\)|\]|\>)) # Tercera Prueba
            #(?!<(?:\<|\"|\[|\()[^)\]\"\>]+)[,|;](?![^(\[\"\<]+(?:\>|\)|\]|\>))
            

            if (re.findall(r'(,|;)', value)):
                
                if ( (name == 'Author') | (name == 'Maintainer') ):
                    
                    value_T = re.sub(r'\(.*?\)\)\)', '', value) # Remueve caracteres dentro de parentesis
                    value_T0 = re.sub(r'\(.*?\)\)', '', value_T)
                    value_T1 = re.sub(r'\([^()]*\)', '', value_T0)
                    value_T2 = re.sub(r'\(.*?\)', '', value_T1)
            
                    value_s = re.split('(?!<(?:\<|\"|\[|\()[^)\]\"\>]+)[,|;](?![^(\[\"\<]+(?:\>|\)|\]|\>))', value_T2)
                else:
                    value_s = re.split('(?!<(?:\<|\"|\[|\()[^)\]\"\>]+)[,|;](?![^(\[\"\<]+(?:\>|\)|\]|\>))', value)
                
                #print(name, value_s)
                data[name] = value_s
            else:
                #print(name, value_T2) 
                if ( (name == 'Author') | (name == 'Maintainer') ):
                    value_T = re.sub(r'\(.*?\)\)\)', '', value) # Remueve caracteres dentro de parentesis
                    value_T0 = re.sub(r'\(.*?\)\)', '', value_T)
                    value_T1 = re.sub(r'\([^()]*\)', '', value_T0)
                    value_T2 = re.sub(r'\(.*?\)', '', value_T1)
                    
                    value_s = re.split('(?!<(?:\<|\"|\[|\()[^)\]\"\>]+)[,|;](?![^(\[\"\<]+(?:\>|\)|\]|\>))', value_T2)

                else:
                    value_s = value.strip()
                    
                data[name] = value_s

                
       
    collection.insert_one(data)
    return data

async def bound_fetch(sem, url, session):
    async with sem:
        await fetch(url, session)

async def fetch_all():
    url = "https://cran.r-project.org/"
    tasks = set()
    async with ClientSession() as session:
        sem = asyncio.Semaphore(MAX_SIM_CONNS)
        #df[df['name']=='gee']['url'].tolist()
        for url_pkg in listPackages:
            task = asyncio.create_task(bound_fetch(sem, url_pkg, session))
            tasks.add(task)
        
            #await asyncio.sleep(2)
        return await asyncio.gather(*tasks)

if __name__ == '__main__':
    start_time = time()
    
    loop = asyncio.get_event_loop()
    #asyncio.run(fetch_all())
    loop.create_task(fetch_all())
    
    end_time = time()
    elapsed_time = end_time - start_time
    
print(f'Elapsed run time: {elapsed_time} seconds')

Elapsed run time: 2.3126602172851562e-05 seconds
